In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load SpaceX launch data
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
df = pd.read_csv(url)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1('SpaceX Launch Dashboard'),

    # Dropdown for Launch Sites
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
        ],
        value='ALL',
        clearable=False
    ),

    # Pie chart for launch success by site
    dcc.Graph(id='success-pie-chart'),

    # Range slider for payload mass
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=100,
        marks={i: str(i) for i in range(0, 10001, 1000)},
        value=[0, 10000]
    ),

    # Scatter plot for success vs. payload mass
    dcc.Graph(id='success-payload-scatter-chart')
])

# Callback for pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Count the number of successful launches for each site
        df_success = df[df['class'] == 1]
        df_grouped = df_success.groupby('Launch Site').size().reset_index(name='count')
        fig = px.pie(df_grouped, values='count', names='Launch Site', 
                     title='Total Successful Launches by Site')
    else:
        df_filtered = df[df['Launch Site'] == selected_site]
        fig = px.pie(df_filtered, names='class', title=f'Success Rate for {selected_site}')
    return fig

# Callback for scatter plot
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('payload-slider', 'value'),
     Input('site-dropdown', 'value')]
)
def update_scatter_chart(payload_range, selected_site):
    df_filtered = df[(df['Payload Mass (kg)'] >= payload_range[0]) &
                     (df['Payload Mass (kg)'] <= payload_range[1])]
    if selected_site == 'ALL':
        # Show comparison across all sites
        fig = px.scatter(df_filtered, x='Payload Mass (kg)', y='class',
                         color='Launch Site',
                         title='Success vs. Payload Mass for All Sites',
                         labels={'class': 'Launch Success'})
    else:
        df_filtered = df_filtered[df_filtered['Launch Site'] == selected_site]
        fig = px.scatter(df_filtered, x='Payload Mass (kg)', y='class',
                         color='Booster Version Category',
                         title=f'Success vs. Payload Mass for {selected_site}',
                         labels={'class': 'Launch Success'})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8061)
